### 📚 **Importação de Bibliotecas**

In [267]:
from math import sqrt
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


### 🟢 **Gram Schmidt**

<div style="text-align: justify;">
A função norm2 é responsável por calcular a norma euclidiana de um vetor A definido na entrada, retornando a distância euclidiana de A à origem do espaço vetorial.

A função gram_schmidt realiza o processo de ortonormalização de uma matriz A utilizando o método de Gram-Schmidt, garantindo que as colunas da matriz resultante Q sejam ortogonais e unitárias. O algoritmo inicia copiando cada coluna de A para Q. Após isso, remove as componentes não ortogonais de cada vetor ao longo das colunas já ortonormalizadas, utilizando projeções vetoriais (sendo que cada projeção é obtida dividindo o produto vetorial entre o vetor atual e o vetor anterior pelo produto vetorial entre o vetor anterior e ele mesmo). A norma Euclidiana de cada vetor é calculada e, com o auxílio da função ``norm2``, o vetor é normalizado para torná-lo unitário. Caso a norma de um vetor seja inferior ao limite de tolerância especificado, a devolve um erro ao usuário, indicando que há dependência linear entre as colunas de A. Ao término do processo, a matriz Q ortonormalizada é retornada.
</div>

In [268]:
def norm2(A):
    """Norma Euclidiana (L2) de um vetor A."""
    return np.sqrt(np.sum(A**2))  # Soma dos quadrados dos elementos e raiz quadrada

def gram_schmidt(A, tol=1e-10):
    """Retorna uma matriz A ortonormalizada usando o processo de Gram-Schmidt."""
    m, n = A.shape  # `m` é o número de linhas e `n` é o número de colunas da matriz A
    Q = np.zeros((m, n))  # Cria uma matriz Q de zeros com as mesmas dimensões de A
    
    for i in range(n):  # Itera sobre as colunas da matriz A
        Q[:, i] = A[:, i]  # Inicializa o vetor da coluna i de Q com o vetor correspondente de A
        
        for j in range(i):  # Itera sobre as colunas anteriores para ortogonalizar
            # Calcula a projeção do vetor Q[:, i] em Q[:, j]
            proj = np.dot(Q[:, j], Q[:, i]) / np.dot(Q[:, j], Q[:, j])  
            Q[:, i] -= proj * Q[:, j]  # Subtrai a projeção para garantir a ortogonalidade
        
        # Calcula a norma do vetor ortogonalizado
        #norma = np.linalg.norm((Q[:, i]))
        norma = norm2(Q[:, i])
        if norma < tol:  # Verifica se a norma do vetor é suficientemente pequena
            raise ValueError(f"O vetor {i} resultou em uma norma muito pequena ({norma}). Isso indica que as colunas de A são linearmente dependentes.")
        Q[:, i] /= norma  # Normaliza o vetor para torná-lo unitário
    
    return Q  # Retorna a matriz Q com base ortonormal


### 🔴 **Decomposição QR**

<div style="text-align: justify;">
A função decomposicao_qr(A) realiza a decomposição QR de uma matriz A, retornando duas matrizes: Q, que é ortonormal, e R, que é triangular superior. Primeiramente, utiliza a função "gram_schmidt" para obter a matriz Q ortonormalizada. Em seguida, constrói R, onde os elementos da diagonal (R[i,i]) representam a magnitude da projeção de cada coluna de A na direção correspondente de Q, enquanto os elementos acima da diagonal R[j,i], com (j<i) indicam a contribuição das colunas de Q na composição das colunas de A. A partir desse processo, a matriz R é calculada por meio do produto escalar entre as colunas de Q e A e o resultado é retornado como um par de matrizes Q e R.
</div>

In [269]:
def decomposicao_qr(A):
    m, n = A.shape
    Q = gram_schmidt(A)  # Obtenção da matriz Q ortonormal
    R = np.zeros((n, n))  # Inicialização da matriz R
    
    for i in range(n):
        R[i, i] = np.dot(Q[:, i], A[:, i])  # Coeficiente de projeção de ai na direção qi
        for j in range(i):
            R[j, i] = np.dot(Q[:, j], A[:, i])  # Coeficiente da projeção de ai na direção qj
            
    return Q, R

### 🔵 **Método de Francis**

In [270]:
def metodo_francis(A, tolerancia):
    """A matriz A é decomposta pelo método QR até que a matriz A se torne diagonal, facilitando a 
    obtenção dos autovalores e autovetores."""
    n = np.shape(A)[0]
    V = np.eye(n)  # Inicializa uma matriz identidade de dimensão n
    
    erro = float("inf")  # Inicializa o erro como infinito
    
    while erro > tolerancia:  # Realiza a decomposição QR até atingir a tolerância
        Q, R = decomposicao_qr(A)  # Decomposição QR
        A = R @ Q  # Atualiza A com o produto de R e Q
        V = V @ Q  # Atualiza a matriz dos autovetores
        erro = np.max(np.abs(np.tril(A, k=-1)))  # Erro é o maior valor na parte triangular inferior
    
    autovalores = np.diag(A)  # Os autovalores estão na diagonal de A
    autovetores = V  # Os autovetores estão em V
    
    return autovalores, autovetores
    

### 🟡 **PCA**

<div style="text-align: justify ;">
A função PCA, de maneira geral, realiza a Análise de Componentes Principais em um conjunto de dados fornecido na entrada, utilizando a função "algoritmo Francis" para calcular os autovalores e autovetores da matriz de covariância (obtida pela função "cov" da biblioteca Numpy aplicada na matriz original transposta). Como sequência de passos, temos que essa função centraliza os dados por meio da média, calcula a matriz de covariância, ordena os autovalores em ordem decrescente e constroi uma matriz de cargas para os componentes principais (PCs). A partir disso, seleciona um número especificado de componentes principais (dado na entrada pela variável "quant_pcs") e projeta os dados originais no novo espaço reduzido. Ao final, o código retorna a matriz de cargas reduzida e os dados transformados nos componentes principais.
</div>


In [271]:
def PCA(dataframe, atributos, quant_pcs):
    """Realiza a Análise de Componentes Principais (PCA) utilizando o método QR (Francis) para 
    calcular os autovalores e autovetores da matriz de covariância."""
    # Seleção de atributos (sendo atributos uma lista!)
    dados = dataframe[atributos]
    
    # Centralização dos dados
    media = np.mean(dados, axis=0)  
    dados_centralizados = dados - media  
    
    # Obtenção da matriz de covariância entre as variáveis
    matriz_cov = np.cov(dados_centralizados.T)  
    
    # Cálculo de autovalores e autovetores da matriz de covariância pelo Método de Francis
    autovalores, autovetores = metodo_francis(matriz_cov, 1e-10)
    
    # Ordenação dos autovalores em ordem decrescente para a criação de uma matriz de cargas
    indices_ordenados = np.argsort(autovalores)[::-1]
    matriz_de_cargas = autovetores[:, indices_ordenados]
    
    # Criação do df de matriz de cargas
    nomes_componentes = [f"PC{i+1}" for i in range(len(matriz_de_cargas))]
    
    df_matriz_cargas = pd.DataFrame(
        matriz_de_cargas,
        columns=nomes_componentes,
        index=atributos
    )
    
    # Seleção dos componentes principais (mediante a quantidade de PC´s)
    pcs_selecionadas = nomes_componentes[:quant_pcs]
    matriz_cargas_reduzida = df_matriz_cargas[pcs_selecionadas]
    
    # Projeção dos dados nos componentes principais
    X_reduzido = np.dot(dados_centralizados, matriz_cargas_reduzida)
    
    return matriz_cargas_reduzida, X_reduzido


### ⚪ **Aplicação do PCA**

Com a finalidade de testar na prática como a análise de componentes principais funciona, utilizaremos o dataset "penguins" da biblioteca seaborn para diminuir a dimensionalidade dos dados.

In [272]:
dataset = sns.load_dataset("penguins")
dataset_limpo = dataset.dropna()
display(dataset_limpo)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


Como atributos, serão utilizadas três colunas numéricas ('bill_length_mm', 'bill_depth_mm', 'flipper_length_mm'), enquanto para o target será definida a coluna "species" (espécies) 

In [273]:
atributos = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']
dataset_atributos = dataset_limpo[atributos]

Para visualizar os dados, é possível plotar um gráfico em 3D

In [274]:
df = dataset_atributos.copy()
df["species"] = dataset["species"]

fig = px.scatter_3d(
    df,
    x="bill_length_mm",
    y="bill_depth_mm",
    z="flipper_length_mm",
    color="species", 
    title="Espécies de pinguim e suas características",
    labels={
        "bill_length_mm": "Comprimento do Bico (mm)",
        "bill_depth_mm": "Profundidade do Bico (mm)",
        "flipper_length_mm": "Comprimento da Nadadeira (mm)",
        "species": "Espécie",
    }
)

fig.update_traces(marker=dict(size=5)) 
fig.update_layout(
    scene=dict(
        xaxis_title="c_bico (mm)",
        yaxis_title="peso (g)",
        zaxis_title="c_nadadeira (mm)",
    )
)

fig.show()


Ainda é possível visualizar os dados de forma 2D ao escolher dois atributos.

In [275]:
fig = px.scatter(dataset, x="flipper_length_mm", y="bill_depth_mm", color="species")
fig.show()

Visualmente, por essas duas características, é possível perceber que os pontos referentes aos pinguins "Adelie" e "Chinstrap" estão misturados, o que dificulta a visualização dos dados e pode prejudicar o desempenho de um modelo preditivo. Para isso, podemos tentar aplicar a técnica de PCA.

*Treinamento do modelo*

O modelo escolhido foi o K-Nearest-Neighbors (K-nn), o qual se baseia na distância entre o objeto de entrada e os que foram previamente utilizados no treinamento do modelo (vizinhos) em certo plano. A partir disso, ele realiza suas predições com base na média (em caso de um regressor) ou na moda (classificador). Para os testes, o dataset original será dividido em treino (treinamento do modelo) e teste (validação do desempenho do modelo). 

In [276]:
semente_aleatoria = 10101118
tamanho_teste = 0.1

indices = dataset_limpo.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=tamanho_teste, random_state=semente_aleatoria, shuffle=True
)

In [277]:
df_treino = dataset_limpo.loc[indices_treino]
df_teste = dataset_limpo.loc[indices_teste]

In [278]:
df_treino_target = df_treino.reindex(["species"], axis=1)
df_treino_features = df_treino.reindex(atributos, axis=1)

df_teste_target = df_treino.reindex(["species"], axis=1)
df_teste_features = df_treino.reindex(atributos, axis=1)

In [279]:
X_treino = df_treino_features.values
y_treino = df_treino_target.values.ravel()

X_teste = df_teste_features.values
y_teste = df_teste_target.values.ravel()

*Treinamento do modelo sem PCA*

In [290]:
modelo_knn = KNeighborsClassifier()

modelo_knn.fit(X_treino, y_treino)

y_pred_linear = modelo_knn.predict(X_teste)

scores_knn= cross_val_score(modelo_knn, X_treino, y_treino, cv=10, scoring="accuracy")
acuracia_knn = np.mean(scores_knn)
print(f"A acurácia do modelo K-nn foi de {acuracia_knn}")

A acurácia do modelo K-nn foi de 0.9598850574712643


*Treinamento do modelo com PCA*

In [281]:
matriz_carga_treino, X_reduzido_treino = PCA(df_treino_features, atributos, 2)
matriz_carga_teste, X_reduzido_teste = PCA(df_teste_features, atributos, 2)

In [291]:
modelo_knn_pca = KNeighborsClassifier()

modelo_knn_pca.fit(X_reduzido_treino, y_treino)

y_pred_knn_pca = modelo_knn_pca.predict(X_reduzido_teste)

scores_knn_pca = cross_val_score(modelo_knn_pca, X_reduzido_treino, y_treino, cv=10, scoring="accuracy")
acuracia_knn_pca = np.mean(scores_knn_pca)
print(f"A acurácia do modelo K-nn com PCA foi de {acuracia_knn_pca}")

A acurácia do modelo K-nn com PCA foi de 0.953103448275862


A partir da comparação dos resultados de acurácia dos modelos, foi possível concluir que não houve diferença significativa no desempenho após a implementação da análise de componentes principais.

No entanto, como evidenciado pelo gráfico abaixo, a aplicação de PCA melhorou a visualização da separação entre as espécies em um plano 2D.

In [289]:
matriz_carga_completa, X_reduzido_completa = PCA(dataset_atributos, atributos, 2)
df_pca = pd.DataFrame(X_reduzido_completa, columns=["PC1", "PC2"])
df_pca["species"] = dataset["species"]
fig = px.scatter(df_pca, x="PC1", y="PC2", color="species", title="Espécie de penguins com PCA")
fig.show()

### 🗃️ **Referências!**

[1] David S. Watkins, Fundamentals of Matrix Computations, John Wiley & Sons, 1991.

[2] Daniel Roberto Cassar. (2024). Jupyter Notebook ATP-203 8.0 - A matrix de covariância e a matriz de correlação. [Material não publicado].

[3] Daniel Roberto Cassar. (2024). Jupyter Notebook ATP-203 8.1 - Redução de dimensionalidade com PCA. [Material não publicado].